In [1]:
import numpy as np
import pandas as pd

# Load CSV files.
#CSV task A 
def getData():
    df_train_data = pd.read_csv("data/Training_Data/subtaskA_data_all.csv")
    df_train_answers = pd.read_csv("data/Training_Data/subtaskA_answers_all.csv")

    df_train = pd.merge(df_train_data,df_train_answers,on='id', how='left').drop(['id'], axis=1)
    
    df_dev_data = pd.read_csv("data/Dev_Data/subtaskA_dev_data.csv")
    df_dev_answers = pd.read_csv("data/Dev_Data/subtaskA_gold_answers.csv")

    df_dev = pd.merge(df_dev_data,df_dev_answers,on='id', how='left').drop(['id'], axis=1)

    df_test_data = pd.read_csv("data/Test_Data/subtaskA_test_data.csv")
    df_test_answers = pd.read_csv("data/Test_Data/subtaskA_gold_answers.csv")

    df_test= pd.merge(df_test_data,df_test_answers,on='id', how='left').drop(['id'], axis=1)
    
    return df_train, df_dev, df_test

df_train_A, df_dev_A, df_test_A = getData()



In [3]:
from transformers import BertModel
from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

In [4]:
model = BertClassifier(max_seq_length=64, train_batch_size=16)
model

Building sklearn text classifier...


BertClassifier(max_seq_length=64, train_batch_size=16)

In [7]:
# subsample data 
n = 1000
train = df_train_A.sample(n, random_state=42)

X_train = train[['sent0', 'sent1']]
y_train = train['answer']

# use the dev set for testing

X_test = df_dev_A[['sent0', 'sent1']]
y_test = df_dev_A['answer']
                                                                                                            

In [8]:
model = model.fit(X_train, y_train)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 900, validation data size: 100


Training  :   0%|                                                                               | 0/57 [00:00<?, ?it/s]C:\Users\ellyn_vdxio7o\miniconda3\envs\tarProject\lib\site-packages\bert_sklearn\model\pytorch_pretrained\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1485.)
  next_m.mul_(beta1).add_(1 - beta1, grad)
Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:50<00:00,  3.87s/it]

Epoch 1, Train loss: 0.7336, Val loss: 0.7343, Val accy: 45.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:49<00:00,  3.79s/it]

Epoch 2, Train loss: 0.6791, Val loss: 0.6698, Val accy: 55.00%



Validating: 100%|██████████████████████████████████████████████████████████████████████| 13/13 [00:39<00:00,  3.04s/it]

Epoch 3, Train loss: 0.4303, Val loss: 0.7001, Val accy: 66.00%


In [9]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression as LR


def test_performance(model, x_test, y_test):
    type(model)
    y_pred = model.predict(x_test)
    print(classification_report(y_pred=y_pred, y_true=y_test))
    return f1_score(y_pred=y_pred, y_true=y_test, average="binary")

In [11]:
f1 = test_performance(model, X_test, y_test)
print(f"f1 = {f1:.3f}")

Predicting: 100%|████████████████████████████████████████████████████████████████████| 125/125 [03:11<00:00,  1.53s/it]

              precision    recall  f1-score   support

           0       0.68      0.57      0.62       518
           1       0.60      0.71      0.65       479

    accuracy                           0.64       997
   macro avg       0.64      0.64      0.64       997
weighted avg       0.64      0.64      0.64       997

f1 = 0.654
